<a href="https://colab.research.google.com/github/vian2298/Mineria-de-datos/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# progress bars 
from tqdm.notebook import tqdm
tqdm.pandas()

# text manipulation
'''
#from unidecode import unidecode
import re
import string
import nltk
nltk.download('punkt')

# gensim - for training embedding models
import gensim
#import gensim.similarities.index
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# topic modeling, etc
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering, KMeans '''

"\n#from unidecode import unidecode\nimport re\nimport string\nimport nltk\nnltk.download('punkt')\n\n# gensim - for training embedding models\nimport gensim\n#import gensim.similarities.index\nimport logging\nlogging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)\n\n# topic modeling, etc\nfrom sklearn.decomposition import PCA\nfrom scipy.cluster.hierarchy import dendrogram\nfrom sklearn.cluster import AgglomerativeClustering, KMeans "

In [ ]:
import seaborn as sns

In [ ]:
def get_twitter_df():
    manifest = pd.read_csv(
        'https://raw.githubusercontent.com/cdcmx2020a/group_a_twitter/master/processed_json/manifest.txt',
        header=None,
        names=['jsons'])['jsons'].tolist()
    df = pd.concat(
        [pd.read_json(f'https://github.com/cdcmx2020a/group_a_twitter/blob/master/processed_json/{f}?raw=true', lines=True) 
            for f in tqdm(manifest)],
        axis=0
    )
    # df = df.query('date > 60')
    return df 

In [ ]:
tweets_es = get_twitter_df().query('lang == "es"')

  0%|          | 0/74 [00:00<?, ?it/s]

In [ ]:
tweets_es

,full_text,lat,lon,date,hashtags,place_type,place_name,country,favorites,retweets,lang
5,7 consejos para que las marcas afronten la cri...,NaN,NaN,78,[],None,None,None,11,3,es
6,RT @marianoemartin: Alejandro Borensztein dice...,NaN,NaN,74,[],None,None,None,0,222,es
8,RT @Icomem_Oficial: Con respecto a las declara...,NaN,NaN,88,[],None,None,None,0,5094,es
11,RT @astrovidenteve: Lo diré nuevamente así me ...,NaN,NaN,80,[],None,None,None,0,89,es
12,Albert Camus vuelve a ser un éxito de ventas. ...,NaN,NaN,67,[],None,None,None,15,5,es
...,...,...,...,...,...,...,...,...,...,...,...
25114,¡Instalan 11 ventiladores más para pacientes C...,NaN,NaN,197,[],None,None,None,1,0,es
25116,"RT @rsuncion54: Todos los que nos contagiamos,...",NaN,NaN,192,[],None,None,None,0,5,es
25117,"RT @ZorlakRules: ""Fallecieron porque para no g...",NaN,NaN,184,[],None,None,None,0,625,es
25130,"RT @hdemauleon: Brutal.\nChoferes, ambulantes,...",NaN,NaN,191,[],None,None,None,0,3487,es


In [ ]:
sentiment=tweets_es[['full_text', 'date', 'retweets', 'place_name']]

In [ ]:
punctuation_chars = ["'", '"', ",", ".", "!", ":", ";", '#', '@']

def strip_punctuation(strr):
    for i in punctuation_chars:
        if i in strr:
            strr=strr.replace(i, "")
    return strr

In [ ]:
sentiment['full_text']=sentiment['full_text'].apply(strip_punctuation)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
sentiment

,full_text,date,retweets,place_name
44542,Me enamoro de coronavirus y se va Listo les sa...,60,0,Benito Juárez
79706,easanchezl checa este hilo https//tco/VJAmrJN6hL,60,0,Bahía de Banderas
59619,El coronavirus llega a cdmx y lo meten en un b...,60,6,Benito Juárez
69316,Oyitessssss HugoRodriguezzp zany_face https//t...,60,0,Hermosillo
99432,Intercambio fluidos con un italiano https//tco...,60,0,Puebla
...,...,...,...,...
14498,Perdió su empleo por COVID-19 y un juez obligó...,198,0,Espita
75038,Ya solo quedan 9 municipios libres de Covid-19...,198,0,Hermosillo
60390,WEY HAY QUE HACER UN RITUAL PARA QUE SE ACABE ...,198,0,Metepec
25123,Mañana regresaría a mí trabajo te odio covid-19,198,0,Tezontepec de Aldama
